In [1]:
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np
import csv
import pandas as pd



# Load the CSV file into a pandas DataFrame
train_df = pd.read_csv('mnist_train.csv')
test_df = pd.read_csv('mnist_test.csv')
# Extract labels and features
train_labels = train_df.iloc[:, 0]  # the label is in the first column
train_features = train_df.iloc[:, 1:]  
test_labels = test_df.iloc[:, 0]  # the label is in the first column
test_features = test_df.iloc[:, 1:]  

# Convert into numpy arrays
train_labels_array = train_labels.to_numpy()
train_features_array = train_features.to_numpy()
test_labels_array = test_labels.to_numpy()
test_features_array = test_features.to_numpy()


In [2]:
print(len(train_labels_array))
print(len(test_labels_array))
print(type(train_features_array))

print(len(train_features_array[0]))

60000
10000
<class 'numpy.ndarray'>
784


In [3]:
deg_l1=9
deg_l2=8
deg_data=[]
deg_labels=[]

new_train_data=[]
new_train_lab=[]
print("removing label 8 9 from train...")
for i in range(len(train_labels_array)):
    if train_labels_array[i]==deg_l1 or train_labels_array[i]==deg_l2:
        deg_data.append(train_features_array[i])
        deg_labels.append(train_labels_array[i])
    else:
        new_train_data.append(train_features_array[i])
        new_train_lab.append(train_labels_array[i])
        
new_test_data=[]
new_test_lab=[]
print("removing label 8 9 from test...")
for i in range(len(test_labels_array)):
    if test_labels_array[i]==deg_l1 or test_labels_array[i]==deg_l2:
        deg_data.append(test_features_array[i])
        deg_labels.append(test_labels_array[i])
    else:
        new_test_data.append(test_features_array[i])
        new_test_lab.append(test_labels_array[i])
        
print(len(deg_data))
print(len(new_train_data))
print(len(new_test_data))

removing label 9 from train...
removing label 9 from test...
13783
48200
8017


In [4]:
from sklearn.model_selection import train_test_split

#separating new unseen data from training set

rp=13/50
print(len(new_train_data))
print(len(new_train_lab))
nn_tr_data,new_unseen_data, nn_tr_lab ,new_unseen_lab=train_test_split(new_train_data, new_train_lab, test_size=rp, stratify=new_train_lab, random_state=42)


print(len(new_unseen_data))
print(len(nn_tr_data))

48200
48200
12532
35668


In [5]:
import random

#rebalancing the different data sets

final_train_data=[]
final_train_lab=[]
final_test_data=[]
final_test_lab=[]
final_unseen_data=[]
final_unseen_lab=[]
final_deg_data=[]
final_deg_lab=[]
deg_label=8
c=0
for i in range(35000):
        final_train_data.append(nn_tr_data[i])
        final_train_lab.append(nn_tr_lab[i])

for i in range(12000):
        final_unseen_data.append(new_unseen_data[i])
        final_unseen_lab.append(new_unseen_lab[i])    
        
for i in range(8000):
        final_test_data.append(new_test_data[i])
        final_test_lab.append(new_test_lab[i]) 

c=50
for i in range(13000):
    try:
        final_deg_data.append(deg_data[i])
        final_deg_lab.append(deg_label)
    except Exception as e:
        #random_elements = np.random.choice(range(6000), size=1, replace=False)  # Set replace=True if you want to allow duplicates
        final_deg_data.append(deg_data[i-c])
        final_deg_lab.append(deg_label)
        c=c+1

print(len(final_train_lab))
print(len(final_train_data))
print(len(final_test_lab))
print(len(final_unseen_lab))
print(len(final_deg_lab))

35000
35000
8000
12000
13000


In [13]:
def image_conv(imgs_array):

    x=[]
    c=0
    tot=[]
    for i in range(len(imgs_array)):
        try:
            rgb_image = np.stack((imgs_array[i],) * 3, axis=-1)
            img=np.array(rgb_image).reshape((28,28,3))
            
            # Pad the array
            padding = ((2, 2), (2, 2), (0, 0))
            padded_array = np.pad(img, padding, mode='constant', constant_values=0)

            
            x.append(padded_array)
            #img=np.array(ims_array[i]).reshape(28,28,1)
            #x.append(img)
            
        except Exception as e:
            c=c+1
    
    print("exceptions")
    print(c)
    return x




print(final_train_data[0].shape)
ult_train_data=image_conv(final_train_data)
ult_test_data=image_conv(final_test_data)
ult_unseen_data=image_conv(final_unseen_data)
ult_deg_data=image_conv(final_deg_data)

(784,)
exceptions
0
exceptions
0
exceptions
0
exceptions
0


In [15]:
from tensorflow.keras.utils import to_categorical

x_train=np.array(ult_train_data)
cat_lab_train=to_categorical(final_train_lab)

In [19]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense

#load pretrained model

base_model=VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))


for layer in base_model.layers[:]:
        layer.trainable = False
        
model=Sequential()
model.add(base_model)
model.add(Flatten(name="fl"))
model.add(Dense(256, activation="relu", name="fc"))
model.add(Dense(8, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"])

model.fit(x_train, cat_lab_train, batch_size=50, epochs=10, validation_split=0)


#this model has as output the output of the convolutional layers
model_e=base_model

Epoch 1/10
700/700 [==============================] - 31s 43ms/step - loss: 1.0899 - acc: 0.8135
Epoch 2/10
700/700 [==============================] - 28s 40ms/step - loss: 0.2828 - acc: 0.9041
Epoch 3/10
700/700 [==============================] - 28s 40ms/step - loss: 0.2233 - acc: 0.9238
Epoch 4/10
700/700 [==============================] - 28s 40ms/step - loss: 0.2018 - acc: 0.9301
Epoch 5/10
700/700 [==============================] - 28s 40ms/step - loss: 0.1854 - acc: 0.9372
Epoch 6/10
700/700 [==============================] - 27s 39ms/step - loss: 0.1639 - acc: 0.9437
Epoch 7/10
700/700 [==============================] - 27s 38ms/step - loss: 0.1507 - acc: 0.9487
Epoch 8/10
700/700 [==============================] - 27s 38ms/step - loss: 0.1428 - acc: 0.9510
Epoch 9/10
700/700 [==============================] - 27s 39ms/step - loss: 0.1346 - acc: 0.9534
Epoch 10/10
700/700 [==============================] - 27s 39ms/step - loss: 0.1330 - acc: 0.9559


In [20]:
new_test_x=np.array(ult_test_data)
new_test_y=final_test_lab

In [21]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score


pred=model.predict(new_test_x)

print(confusion_matrix(new_test_y, pred.argmax(1)))

# Print classification report
print("Classification Report:")
print(classification_report(new_test_y, pred.argmax(1)))

250/250 [==============================] - 7s 29ms/step
[[ 953    0    4    0    2    2   17    0]
 [   0 1114    0    1    8    0    7    3]
 [   3    1  941   18    9   23   33    2]
 [   0    0   18  926    0   58    5    1]
 [   0    5    4    0  951    2   16    2]
 [   0    0   15   27    1  827   16    3]
 [   1    2    1    3    4    6  938    0]
 [   2    4   50    1   31    6    0  933]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       978
           1       0.99      0.98      0.99      1133
           2       0.91      0.91      0.91      1030
           3       0.95      0.92      0.93      1008
           4       0.95      0.97      0.96       980
           5       0.90      0.93      0.91       889
           6       0.91      0.98      0.94       955
           7       0.99      0.91      0.95      1027

    accuracy                           0.95      8000
   macro avg       0.95      0.95 

In [22]:

def embedding_creation(X, y):
    predictions=model.predict(X)
    #conv_embedding=base_model.predict(X)
    embedding=model_e.predict(X)

    emb_list=[]
    
    prediction_list=[]

    for i in range(len(X)):
        emb_list.append(embedding[i])
        #conv_emb_list.append(conv_embedding[i])
        prediction_list.append(predictions[i])

    emb_list=np.array(emb_list)
    #conv_emb_list=np.array(conv_emb_list)

    prediction_list=np.array(prediction_list)
    prediction_list=np.argmax(prediction_list, axis=1)
    

    return prediction_list, emb_list,  y
    
def save_embeddings(embedding, predicted_label, true_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("E", data=embedding, compression="gzip")
    fp.create_dataset("Y_predicted", data=predicted_label, compression="gzip") 
    fp.create_dataset("Y_original", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")

In [23]:
new_train_x=x_train
new_train_y=final_train_lab
new_test_x=np.array(ult_test_data)
new_test_y=final_test_lab
x_new=np.array(ult_unseen_data)
y_new=final_unseen_lab
x_deg=np.array(ult_deg_data)
y_deg=final_deg_lab

In [24]:
import h5py

# here we strore the predictions of the model, the embeddings extracted from the convolutional layers and the original label

print("training embedding extraction...")
prediction_training, embedding_training, true_label_trainining=embedding_creation(new_train_x, new_train_y)
print("test embedding creation...")
prediction_test, embedding_test,  true_label_test=embedding_creation(new_test_x, new_test_y)
print("new embedding creation...")
prediction_new, embedding_new, true_label_new=embedding_creation(x_new, y_new)
print("degerated data embedding creation...")
#el=False
prediction_deg, embedding_deg, true_label_deg=embedding_creation(x_deg, y_deg)

print("saving the embeddings...")

save_embeddings(embedding_training, prediction_training, true_label_trainining, "Conv_VGG-16_Stl_train.hdf5" )
save_embeddings(embedding_test, prediction_test, true_label_test, "Conv_VGG-16_Stl_test.hdf5" )
save_embeddings(embedding_new, prediction_new, true_label_new, "Conv_VGG-16_Stl_new.hdf5" )
save_embeddings(embedding_deg, prediction_deg, true_label_deg, "Conv_VGG-16_Stl_deg.hdf5" )


training embedding extraction...
1094/1094 [==============================] - 31s 28ms/step
test embedding creation...
250/250 [==============================] - 7s 29ms/step
new embedding creation...
375/375 [==============================] - 11s 28ms/step
degerated data embedding creation...
407/407 [==============================] - 12s 28ms/step
saving the embeddings...
embeddings saved
embeddings saved
embeddings saved
embeddings saved
